In [ ]:
#from command line run: conda activate taccari_ext_env
#pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#pip install torch-sparse  -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#pip install torch-cluster -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
#pip install torch-geometric

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import sklearn
from pathlib import Path

In [ ]:
#run data_processing - DMG added False as a default for synthetic data since I don't have that.
from data_preprocessing import process_data as process_data

In [ ]:
train_data, val_data, test_data, train_mask, val_mask, test_mask, df_piezo_columns, pump_columns, scaler = process_data.main()

In [ ]:
from data_preprocessing import gnn_data_prep as gnp

In [42]:
adj, feats = gnp.main(
    df_piezo_columns,
    pump_columns,
    distance_csv="C:\\Users\\danielg\\PycharmProjects\\Taccari_et_al\\GroundwaterFlowGNN-main\\data\\input\\wells\\wellfield_to_obswell_distances.csv",
    piezo_metadata_csv="C:\\Users\\danielg\\PycharmProjects\\Taccari_et_al\\GroundwaterFlowGNN-main\\data\\preprocessed\\well_metadata.csv",
    percentage=100,
    n_piezo_connected=3
)

In [43]:
import torch
import torch.optim as optim
import torch.nn as nn

from models.train_model import (
    create_mtgnn_model,
    train,
    define_base_configuration
)

from utils.metrics import (
    calculate_rmse_per_piezometer,
)

from utils.training_utils import (
    make_predictions
)

from data_preprocessing.dataset import AutoregressiveTimeSeriesDataset

# 1) Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = define_base_configuration()

num_nodes = int(feats.shape[0])      # N, a Python int
num_piezo = int(len(df_piezo_columns))
num_pump  = num_nodes - num_piezo    # also an int
num_feats  = int(feats.shape[1])
W          = config['W']
F_w        = 3  # or whatever future window

In [44]:

# 2) Build model, optimizer, loss
model = create_mtgnn_model(
    num_features=num_feats,
    num_nodes=num_nodes,
    seq_length=W,
    model_type='MTGNN',
    **config
).to(device)

optimizer     = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()

In [ ]:

# 3) Train
train(
    model, optimizer, loss_function, device,
    num_epochs=200,
    train_data=train_data,
    val_data=val_data,
    train_mask=train_mask,
    val_mask=val_mask,
    df_piezo_columns=df_piezo_columns,
    num_piezo=num_piezo,
    static_features=feats,
    A_tilde=adj,
    F_w=F_w,
    W=W,
    config=config,
    model_type='MTGNN'
)

In [ ]:

# 4) Evaluate on one test sample
_, _, _, mask_seq = AutoregressiveTimeSeriesDataset(
    test_data, W, F_w, test_mask, num_piezo
 )[1]

test_input, test_pred, test_target = make_predictions(
    model, (None, None, None, mask_seq), device, F_w, W,
    A_tilde, static_features, num_piezo, modeltype='MTGNN'
)

In [33]:

# 5) Inverse-scale & compute RMSE
test_pred_ = inverse_transform_with_shape_adjustment(test_pred.numpy(), scaler, num_piezo)
test_targ_ = inverse_transform_with_shape_adjustment(test_target.numpy(), scaler, num_piezo)

rmse_per_pz = calculate_rmse_per_piezometer(test_pred_, test_targ_, num_piezo)
mean, std = rmse_per_pz.mean(), rmse_per_pz.std()
print(f"Test RMSE = {mean:.3f} ± {std:.3f}")


ValueError: only one element tensors can be converted to Python scalars